**<h1>Group Project</h1>**

**<h1>Lyrics to singer classification</h1>**
---

> 

   **<h2>Մասնակիցներ</h2>**


*   Հրանուշ Սահրադյան
*   Հասմիկ Մնացականյան
*   Մերի Վարդանյան
*   Արմեն Մանուկյան


>






**<h2>Purpose of the project</h2>**

Նախագծի նպատակն է համացանցից հավաքել (web scraping-ի միջոցով) 20 տարբեր հայ երգիչների երգերի բառեր, դրանք մշակելով ստանալ տվյալների բազմություն, բաժանել մի քանի մասի (train, val, test) և այդ տվյալների բազմության վրա ստանալ այնպիսի մոդել, որի միջոցով հնարավոր կլինի ինչ֊որ ճշտությամբ գուշակել, թե որ երգիչներին են պատկանում մոդելի կողմից չտեսնված երգերի բառերը։











**<h2>Contents</h2>**


1.   [Data Cleaning and Preprocessing](#preproc)
3.   [Feature Selection/Extraction](#fenigine)
4.   [EDA](#eda)
5.   [Model Selection](**https://**)
6.   [Model Training/Evaluation](**https://**)
7.   [Hyperparameter Tuning](**https://**)
8.   [Model Validation](**https://**)
9.   [Model Deployment and Monitoring](**https://**)

**Note**: Data collection part you can see in the notebook "Data collection"

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time 
import glob
import requests
import nltk
from langdetect import detect_langs

 <a name="preproc"></a>

## Data Cleaning and Preprocessing

Data cleaning and preprocessing for a dataset on Armenian artists and their song lyrics typically involve several steps to ensure the data is accurate, consistent, and ready for analysis or modeling. Here's an overview of the potential steps involved:

**Handling Missing Values:** Identify any missing values in the dataset, such as empty cells or null values. Depending on the context, you can choose to either remove rows or columns with missing values, or impute the missing values using techniques like mean, median, or mode imputation. Consider the impact of missing values on variables like artist names, song titles, and lyrics, and handle them accordingly.

**Removing Duplicate Entries:** Check for and remove any duplicate entries in the dataset. Duplicates can arise from data collection or merging processes and can introduce bias or affect analysis results. Identify duplicate rows or combinations of variables and remove them to maintain data integrity.

**Cleaning and Standardizing Textual Data:** Perform text cleaning and preprocessing on variables like artist names, song titles, and lyrics. This may involve steps such as removing special characters, punctuation, or unnecessary whitespace, converting text to lowercase or uppercase consistently, and handling encoding issues if any.

**Language Detection:** Since you are interested in Armenian lyrics, you can apply language detection techniques to identify whether the lyrics are in Armenian or another language. This step ensures that you are working with the desired language and can filter out any non-Armenian lyrics if needed.

**Dealing with Stop Words and Noise:** In the lyrics column, consider removing common stop words (e.g., "the," "and," "is") that do not carry substantial meaning. Additionally, address any noise or irrelevant text that may be present in the lyrics, such as ad-libs or non-lyrical portions.

**Tokenization and Lemmatization:** Split the lyrics into individual words or tokens using tokenization techniques. This allows for further analysis or modeling on a word-level basis. Additionally, consider applying lemmatization to reduce words to their base or root form, improving consistency and reducing dimensionality.

**Handling Non-Textual Data:** If your dataset contains non-textual data, such as artist metadata or genre information, ensure appropriate cleaning and preprocessing. This may involve transforming categorical variables using techniques like one-hot encoding or label encoding to make them compatible with analysis or modeling techniques.

**Data Validation and Sanity Checks:** Validate the dataset against predefined constraints or business rules to ensure data integrity. Perform sanity checks to identify any inconsistencies, outliers, or unexpected patterns. This step helps ensure the dataset aligns with domain knowledge and meets your expectations.


In [5]:
df=pd.read_csv("FinalData.csv")

Let's see what we have in our dataset.

In [6]:
df.shape

(693, 3)

In [7]:
df.groupby('Artist').count()

,Song,Lyrics
Artist,,
Armenchik,13,13
Lav Eli,10,10
Aram MP3,28,28
Arame,19,19
Arsen Safaryan,28,28
CKK,5,5
Cris,1,1
Empyray,11,11
Enomenal,1,1


In [ ]:
print(df['Lyrics'].iloc[1033])

   Մեզ թվումա, որ մեզ թվումա  Որ մեկը մեզ հետևումա, մեկը մեր հետևումա  Մեր մտքերը գողանումա, տանում  Ես լավանում եմ, դու՝ վատանում  Մենք որոշում ենք, հետո փոշմանում  Ասում ենք երկար ու երկար չենք անում  Ու եթե մի 10 հոգի էս պահին ներս գային  Լավ կլիներ մեր ռիթմերը զգային  Էս պահին, տես գահին  Մեջը կորած ասեղ, եթե գտնես քեզ պահի  Մատդ ծակի կասես, հանրագիտարանը բաց  Ու «Ա» տառից սկսի էս պահին  Ինչ կա շատ, հավասար կիսի  Անցնում էր լիմուզինը, դե սպանի  Մի գնդակով 2 ճագար չես սպանի  Ու եթե հոգի կա օդում մի քանի  Ուրեմն Էս պահին արթնացի մի քնի     Էս պահին չեմ ուզում էլ նայեմ ժամին  Ինձ թողեք մենակ ես սիրում եմ քամին  Ու ոչ մեկ չգիտի, որ քամին գույն ունի  Ու նայում է հեռվից էս պահին  Էս պահին չեմ ուզում էլ նայեմ ժամին  Ինձ թողեք մենակ ես սիրում եմ քամին  Ու ոչ մեկ չգիտի, որ քամին գույն ունի  Ու նայում է հեռվից էս պահին 


It looks like the texts are formated in different styles: the first example includes text in round brackets which says how often parts of the text shall be repeated e.g. "(repeat previous line 3 times)".

Looks like the text in round brackets does mostly not belong to the lyrics of the song and should be removed.

In [ ]:
text_in_round_brackets = sum(list(df['Lyrics'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(text_in_round_brackets)))

Number of round brackets: 14


In [ ]:
random.seed(0)
random.choices(text_in_round_brackets, k=20)

["Put your hands up if you're here to party Have the best party, let's go",
 'Մրոն շամշին բացեր ա Թևաս մաներ գացեր ա',
 'Ты моя моя, нежная моя, Ты моя, моя',
 'Մրոն շամշին բացեր ա Թևաս մաներ գացեր ա',
 'Զարդ ու զարդարանք, Ի՞նչ է պետք, դուն սիրուն ղուշ ես, Մարմինդ է զուգված, ալ խալերով, Դրախտի նուշ ես:  Սիրուն, սև աչերուդ, Կարմիր վարդ այտերուդ, Շանթ ունքերուդ, Շուխ մազերուդ!!! Էլ ի՞նչ սուրմա է, հարկավոր է, Աննման փերուդ:  Ափսոս ջահել ես, Նոր ես ծաղկել, փնջիկ ես գարնան, Քեզ ջան եմ տալիս, չեմ հասկանում, Անգութ հոգեհան:',
 'Ты моя моя, нежная моя, Ты моя, моя',
 'Մրոն շամշին բացեր ա Թևաս մաներ գացեր ա',
 "Put your hands up if you're here to party Have the best party, let's go",
 'Ты моя моя, нежная моя, Ты моя, моя',
 'Չարեգանց պզդիգ թոռը Ղուլբեն ընգել ա հորը',
 'Ты моя моя, нежная моя, Ты моя, моя',
 'Զարդ ու զարդարանք, Ի՞նչ է պետք, դուն սիրուն ղուշ ես, Մարմինդ է զուգված, ալ խալերով, Դրախտի նուշ ես:  Սիրուն, սև աչերուդ, Կարմիր վարդ այտերուդ, Շանթ ունքերուդ, Շուխ մազերուդ!!! Էլ ի՞նչ սուրմ

Removeing round/square brackest and text within.

In [ ]:
df['Lyrics'] = df['Lyrics'].map(lambda s: re.sub(r'\((.*?)\)', '', s))
df['Lyrics'] = df['Lyrics'].map(lambda s: re.sub(r'\[(.*?)\]', '', s))

As we can see all the songs written in Armenian within our dataset, so let's just remove any non-Armenian words present in the lyrics.

In [ ]:
armenian_pattern = re.compile(r'[Ա-Ֆա-ֆև\s]+')
df['Lyrics'] = df['Lyrics'].apply(lambda x: ''.join(armenian_pattern.findall(x)))

df = df[df['Lyrics'].str.strip().astype(bool)]

df.reset_index(drop=True, inplace=True)


Let's get rid of duplicate songs too.

In [ ]:
duplicates = df.duplicated(subset='Song')

# Check if there are any duplicates
if duplicates.any():
    # Remove the duplicate rows from the DataFrame
    df = df.drop_duplicates(subset='Song', keep='first').reset_index(drop=True)
    print("Duplicate values removed.")
else:
    print("No duplicate values found in the 'Song' column.")


Duplicate values removed.


In order to classify the 20 Armenian artists with the most songs in our dataset, we will keep only the records of those 20 artists in our dataframe.

In [ ]:
artist_counts = df.groupby('Artist').count()
top_artists = artist_counts.nlargest(20, 'Song').index
df = df[df['Artist'].isin(top_artists)].reset_index(drop=True)

What we have after removing non armenian and duplicate songs for top 20 Artists in our dataset.

In [ ]:
df.groupby('Artist').count()

,Song,Lyrics,lines
Artist,,,
Lav Eli,10,10,10
Aram MP3,18,18,18
Arame,11,11,11
Arsen Safaryan,24,24,24
Empyray,11,11,11
HT Hayko,16,16,16
Harout Pamboukjian,40,40,40
Mi Qani Hogi,16,16,16
Misho,29,29,29


In [ ]:
df.shape

(543, 4)

<a name="fenigine"> </a>

## Feature Selection/Extraction

### Translating the obtained data to English 

In [ ]:
!pip install googletrans

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from googletrans import Translator

translator = Translator()
error_indices = []

def translate_to_english(text):
    if text is not None:
        try:
            translation = translator.translate(text,src='hy', dest='en')
            return translation.text
        except Exception as e:
            print(f"Translation error: {e}")
            error_indices.append(df.index[df['Lyrics'] == text][0])
    return ''

df['Translated_Lyrics'] = df['Lyrics'].apply(translate_to_english)

print('Indices of rows with translation errors: {}'.format(error_indices))

Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translation error: 'NoneType' object has no attribute 'group'
Translat

KeyboardInterrupt: ignored

In [ ]:
df=pd.read_csv("FinalData1.csv")

In [ ]:
df=df.dropna()

In [ ]:
df.groupby('Artist').count()

,Song,Lyrics,Translated_Lyrics
Artist,,,
Lav Eli,9,9,9
Aram MP3,17,17,17
Arame,9,9,9
Arsen Safaryan,23,23,23
Empyray,10,10,10
HT Hayko,16,16,16
Harout Pamboukjian,32,32,32
Mi Qani Hogi,15,15,15
Misho,26,26,26


In [ ]:
df['lines'] = df['Lyrics'].map(lambda t: len(re.findall(r'\n', t)))
df['Lyrics'] = df['Lyrics'].map(lambda s: re.sub(r'\n|\n', ' ', s))
df['Translated_Lyrics']=df['Translated_Lyrics'].map(lambda s: re.sub(r'\n|\n', ' ', s))

#### Tokenization
A very common way to analyse text is to seperate it into a list of words which makes it much easier to do further analysis. I'm using nltk.tokenize to do that. Furthermore, all punctuations are removed as well. Below is an example how the text is converted.

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
df['tokens_hy'] = df['Lyrics'].map(tokenizer.tokenize)
df['tokens_en'] = df['Translated_Lyrics'].map(tokenizer.tokenize)


#### Stemming
Stemming is a simple method to convert words to a common base form. For example converting plural to singular or past tense to present tense. This helps to treat words with the same meaning in the same way.

In contrast to lemmatization it uses a very simple method and does not always find the grammatically correct from. However, as stemming is much faster than lemmatization I'm using it here.

Below is an example how different forms of the word "make" are converted. In this case stemming failes to correctly convert the past tense.

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()


In [ ]:
# create dictionary to map tokens their stem
token_to_stem = {}
# initialise word count
token_count = 0
# iterate through all songs
for lst in df['tokens_en']:
    # iterate through all tokens of song
    for token in lst:
        token_count += 1
        # check if token is in dictionary
        if token not in token_to_stem:
            # add token to dictionary
            token_to_stem[token] = stemmer.stem(token)
            
df['stems_en'] = df['tokens_en'].map(lambda lst: [token_to_stem[token] for token in lst])

print('Number of tokens: {}'.format(token_count))
print('Number of unique tokens: {}'.format(len(token_to_stem.keys())))
print('Number of unique stems: {}'.format(len(set(token_to_stem.values()))))

Number of tokens: 89552
Number of unique tokens: 8837
Number of unique stems: 5833


In [ ]:
df['n_stems_en'] = df['stems_en'].map(len)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506 entries, 0 to 542
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Artist             506 non-null    object
 1   Song               506 non-null    object
 2   Lyrics             506 non-null    object
 3   Translated_Lyrics  506 non-null    object
 4   lines              506 non-null    int64 
 5   tokens_hy          506 non-null    object
 6   tokens_en          506 non-null    object
 7   stems_en           506 non-null    object
 8   n_stems_en         506 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 39.5+ KB


<a name="eda"></a>
# EDA
#### Number of songs, artists, and words

In [ ]:
song_count_df = df.groupby('Artist')[['Song']].count()
fig = px.histogram(song_count_df, x='Song', title='Songs per artist', labels={'Song': 'Songs'})
fig.show()

In [ ]:
# words per song
fig = px.histogram(df, x='n_stems_en', title='Words per song')
fig.show()

In [ ]:
fig = px.box(df, x='Artist', y='n_stems_en', title='Word count per song by artist')
fig.show()

In [ ]:
# number of unique stems
df['n_unique_stems'] = df['stems'].map(lambda lst: len(set(lst)))
# ratio of unique stems
song_df['unique_stems_ratio'] = song_df['n_unique_stems'] / song_df['n_stems']

# attach column to selected artists
song_filter_df = song_filter_df.join(song_df['unique_stems_ratio'])